In [1]:
!pip install unzip
!pip install -r requirements.txt
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1281 sha256=520350f15ae9d5cfc16ec57cd103221c829c2d2214c100327e9b76477269f9d9
  Stored in directory: /root/.cache/pip/wheels/3c/4d/b3/ddd83a91322fba02a91898d3b006090d1df1d3b0ad61bd8b36
Successfully built unzip
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of pyldavis to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━

# Introducción

El objetivo principal de los algoritmos de _matching_ es el de, dado un fragmento de texto, encontrar, de entre un conjunto de candidatos, los textos más similares al fragmento orginial.

Como texto podemos pensar tanto en palabras, en pequeñas frases o en documentos enteros.

Podemos pensar en 3 tipos de técnicas de matching:
- **Coincidencia exacta**: ya vimos ejemplos de este tipo al estudiar la **Distancia de Edición**.
    - A nivel de carácter: strings que difieren en caracteres
    - A nivel de token: strings que difieren en palabras
    - Fonéticos: palabras que suenan de manera similar
- **Coincidencia aproximada o difusa**
- **Coincidencia mediante aproximaciones**

| Candidato / Tipo de resultado 	| Exacta 	| Aproximada 	| Transformación 	|
|-	|-	|-	|-	|
| String 	| Comparación de strings 	| Comparación difusa 	| Ontologías 	|
| Categoría 	| Gramáticas 	| Reconocimiento probabilístico 	| Análisis de topics 	|
| Documento 	| - 	| Recuperación de información 	| Traducción automática 	|

# Regular expressions (Regex)

Muy utilizadas (y conocidas) suelen emplearse al limpiar el texto o buscar formatos dentro del texto. A modo introductorio, las expresiones regulares son una forma de finite state automaton.

<img src=http://www.cs.cornell.edu/courses/cs312/2006fa/recitations/images/dfa-examples.gif>

Son grafos que siguen una secuencia que nosotros definimos. Por ejemplo, el grafo de la izquierda, solo podría generar expresiones como ab, abb, abbb, abbbb y así hasta el infinito. El de la derecha, podría generar expresiones como abcb, o abbb, abbbbbb, por ejemplo.

Conceptualmente, las regex _funcionan_ así _por debajo_. Lógicamente cuando las usamos es mucho más fácil, ¿verdad :D?

La definición de estos grafos es posible mediante la [librería de Python re](https://docs.python.org/3/library/re.html), módulo del paquete base de Python dedicado a las expresiones regulares.

Cierto es que no siempre nos hará falta. Algunas veces con un simple _string.replace()_ o _string.find()_ tendremos suficiente. No obstante, para muchas tareas son bastante útiles.

Algunas tareas típicas en las que se utilizan son la búsqueda (y a veces normalización) de emails, urls, numeros de telefono, etc. Solo la extracción es interesante, pero mediante su normalización nos permite reducir la cardinalidad del vocabulario y asociar entidades similares a un mismo alias.

[Regex Online](https://regexr.com/) es uno de los mejores recursos online para visualizar que hacen los regex

Veamos algunos ejemplos.

In [2]:
# Función que nos ayudará a visualizar algunos resultados

from termcolor import colored
def test_pass(ok, text):
    color = 'green' if ok else 'red'
    return colored(text, color)

In [3]:
import re

In [4]:
text = 'Todos los animales son iguales, pero algunos son más iguales que otros'

In [5]:
RE_TEST = re.compile(r'todos')
print(RE_TEST.match(text))

None


In [6]:
RE_TEST = re.compile(r'Todos')
print(RE_TEST.match(text))

<re.Match object; span=(0, 5), match='Todos'>


In [7]:
RE_TEST = re.compile(r'[a-zA-Z]')
print(RE_TEST.match(text))

<re.Match object; span=(0, 1), match='T'>


In [8]:
RE_TEST = re.compile(r'\bTodos\b')
print(RE_TEST.match(text))

<re.Match object; span=(0, 5), match='Todos'>


In [9]:
RE_TEST = re.compile(r'\bTod\b')
print(RE_TEST.match(text))

None


## Obtener un correo electrónico

In [10]:
"""
^ -> start of string
+ -> match 1 or more preceding regex
[^@]+
@[^@]+
\. -> '.'
"""

RE_EMAIL = re.compile('[^@]+@[^@]+\.[^@]+')

In [11]:
emails_list = [
    '@invalid@adress.com',
    'correo_valido@gmail.com',
    'notan@valido@gmail.com',
    'si.valido.david@gmail.com',
    'paginaweb.com',
    'paginaweb.com@paginaweb.com'
]
for email in emails_list:
    if RE_EMAIL.match(email):
        print(True)
        print(test_pass(True, email))
        print('___')
    else:
        print(False)
        print(test_pass(False, email))
        print('___')

False
@invalid@adress.com
___
True
correo_valido@gmail.com
___
False
notan@valido@gmail.com
___
True
si.valido.david@gmail.com
___
False
paginaweb.com
___
True
paginaweb.com@paginaweb.com
___


## Obtener precios

In [12]:
from random import shuffle
import unicodedata

CURRENCIES = ''.join(chr(i) for i in range(0xffff) if unicodedata.category(chr(i)) == 'Sc')
RE_MONEY_GENERAL= re.compile('((\s|^)([\d]*)(\.)?([\d])*([%s]|e|USD|USD\$|U\$D)(\s|$))'
                          '|((\s|^)([%s]|e|USD|USD\$|U\$D)([\d])*(\.)?([\d])*(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)
RE_MONEY_EU= re.compile('((\s|^)([\d]{0,3}([\.][\d]{3})(,[\d]*))([%s]|e|(USD|USD\$|U\$D))(\s|$))'
                     '|((\s|^)([%s]|e|(USD|USD\$|U\$D))([\d]{0,3}([\.][\d]{3})(,[\d]*))(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)
RE_MONEY_EU_INVERSE= re.compile('((\s|^)([\d]{0,3}([,][\d]{3})(\.[\d]*))([%s]|e|(USD|USD\$|U\$D))(\s|$))'
                             '|((\s|^)([%s]|e|(USD|USD\$|U\$D))([\d]{0,3}([,][\d]{3})(\.[\d]*))(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)


In [13]:
correct_currencies = [
    '$20.2',
    '$.2',
    '$0.2',
    '$3433.2',
    '.2$',
    '2.0$',
    '2.$',
    '2.0€',
    '2¥',
    '20USD',
    '20e',
    '20 €',
    '20 usd',
    '€200.123,2',
    '2.134,56$',
    '23232₽',
    '334,222.20€',
    '20U$D',
    '$200']

incorrect_currencies = [
    'asdfsd',
    '$asdasd',
    '23333,444.20€',
    '€34523sdfas',
    '€213.sd',
    '$3vg554.25',
    'expensive',
    'cheap',
    '2342,222.90€'
]

all_currencies = correct_currencies + incorrect_currencies
shuffle(all_currencies)

for currency in all_currencies:
    if RE_MONEY_GENERAL.match(currency) or RE_MONEY_EU.match(currency) or RE_MONEY_EU_INVERSE.match(currency):
        print(test_pass(True, currency))
    else:
        print(test_pass(False, currency))

334,222.20€
2¥
asdfsd
.2$
€34523sdfas
23232₽
20 €
expensive
20 usd
$20.2
2342,222.90€
2.0€
$0.2
20e
$200
$3433.2
2.134,56$
20U$D
$asdasd
2.0$
$3vg554.25
23333,444.20€
€200.123,2
$.2
€213.sd
2.$
20USD
cheap


# DAWG

Lo presentábamos antes de manera muy  rápida, un _Directed Acyclic Word Graph_ (por sus siglas, DAWG), también llamado, _Deterministic Acyclic Finite State Automaton_ (DAFSA), es un tipo de estructura de datos que permite representar datos de tipo texto y realizar consultas.

![image.png](attachment:image.png)

En el grafo generado se distinguen:
- **Nodos**: un carácter / símbolo
- **Vértices**: enlace con el siguiente carácter / símbolo más probable

http://www.wutka.com/dawg.html


## Ejemplos

In [14]:

from utils import load_movie_titles

In [15]:
datasets_path = './'
movie_titles_file = 'films.txt'

In [16]:
movies_titles = load_movie_titles(datasets_path, movie_titles_file)

## Lo creamos

In [19]:

!pip install shell


In [20]:
!pip install DAWG

  Using cached DAWG-0.8.0.tar.gz (371 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for DAWG
  Running setup.py clean for DAWG
Failed to build DAWG
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (DAWG)


In [21]:
from pydawg import DAWG

dawg = DAWG()

for w in sorted(m.title for m in movies_titles):
    dawg.add_word_unchecked(w)

In [22]:
import random
t = random.choice(movies_titles).title
t

'The Getting of Wisdom'

In [23]:
t in dawg

True

## Operaciones

### Búsqueda por prefijo

In [24]:
for m in dawg.find_all('Batman'):
    print(m)

Batman
Batman: The Movie
Batman Returns


### Prefijo más largo

In [25]:
s = 'La guerra de nunca jamás'
pfx = dawg.longest_prefix(s)
print( s[:pfx])

La 


### Búsqueda en una oración

In [26]:
def token_match(dawg, tknlist):
    for n in range(len(tknlist), 0, -1):
        test_str = ' '.join(tknlist[:n])
        if test_str in dawg:
            return test_str

def token_match_all(dawg, utterance):
    tknlist = utterance.split()
    return [token_match(dawg, tknlist[chunk:])
             for chunk in range(len(tknlist))]

In [27]:
token_match_all(dawg, 'Donde echan Batman y Robin esta noche')

[None, None, 'Batman', None, None, None, None]

### Minimal perfect hash

In [28]:
dawg.word2index('Batman')

242

# Distancia entre textos

[Jellyfish](https://jellyfish.readthedocs.io/en/latest/) es una librería que contiene funciones para el cálculo de similitud entre textos. Dicha similitud puede ser á nivel léxico-gráfico (strings) o fonético.


Algoritmos de comparación de strings:

- Levenshtein Distance
- Damerau-Levenshtein Distance
- Jaro Distance
- Jaro-Winkler Distance
- Match Rating Approach Comparison
- Hamming Distance

Algoritmos de encoding fonético:

- American Soundex
- Metaphone
- NYSIIS (New York State Identification and Intelligence System)
- Match Rating Codex


In [29]:
# !pip3 install jellyfish
import jellyfish



https://pypi.org/project/jellyfish/

## Levenshtein

Recordemos: distancia de Edit (Edición) en la que las operaciones permitidas son la inserción, la eliminación y la sustitución.

In [30]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne negro')

0

In [31]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne negor')

2

In [32]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne nego')

1

In [33]:
jellyfish.levenshtein_distance('Cisnee negro', 'Cisne nego')

2

In [34]:
jellyfish.levenshtein_distance('Cisneee negro', 'Cisne nego')

3

## Damerau-Levenshtein

Recordemos: distancia de Edit (Edición) en la que las operaciones permitidas son la inserción, la eliminación y la transposición de 2 caracteres adyacentes.

In [35]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne negro')

0

In [36]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne negor')

1

In [37]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne nego')

1

In [38]:
jellyfish.damerau_levenshtein_distance('Cisnee negro', 'Cisne nego')

2

In [39]:
jellyfish.damerau_levenshtein_distance('Cisneee negro', 'Cisne nego')

3